In [1]:
import numpy as np
import pandas as pd
from scipy.stats import f

### 1. Measurements of stiffness and bending strength for a sample of 30 pieces of a particular grade of lumber are given in ‘lumber.dat’ (described in Table 5.11 of the textbook). Let's temporarily assume that the left and right sides of the table are samples of matched pairs.

In [2]:
lumber = pd.read_csv('lumber.dat', header=None, delim_whitespace=True)
lumber.columns = ['x1','x2']

### a. Perform a paired comparison using the subtraction technique and your python code in #1 of HW4.

In [3]:
# Testing을 위한 데이터 조정

# Lab1과 Lab2 구분하기
lab1 = lumber.loc[0:14,]
lab2 = lumber.loc[15:29,].reset_index(drop = True)
lumber_2 = pd.concat([lab1, lab2], axis = 1)

# Lab1과 Lab2 사이의 변수들간 차이 구하기
d1 = np.array(lab1['x1'] - lab2['x1'])
d2 = np.array(lab1['x2'] - lab2['x2'])
diff = pd.DataFrame({"d1" : d1, "d2" : d2})

In [4]:
# Use code in HW4, #1

def hotelling_t_test(data, u0, alpha):  # u0 : null hypothesis  / # alpha : significance level
    
    # Calculate Hotelling's T square and Test statisitc
    n = len(data)
    p = len(data.columns)
    sample_mean_v = data.mean()
    mean_diff = sample_mean_v - u0
    cov_v = data.cov()
    cov_inv = np.linalg.inv(cov_v)
    T_square = len(data) * (mean_diff.T @ cov_inv @ mean_diff)
    Test_stat = T_square * ((n-p) / ((n-1)*p))
    
    # Calculate critical region
    c_region = f.ppf(1-alpha, p, n-p, loc=0, scale=1)
    
    # p-value
    p_value = f.sf(Test_stat, p, n-p)
    
    # Hypothesis Testing
    if Test_stat > c_region :
        print("Test Statistic = {:.3f}".format(Test_stat))
        print("p-value = {}".format(p_value))
        print("Hotelling's T^2 = {:.3f}".format(T_square))
        print("Reject the null hypothesis")
    else :
        print("Test Statistic = {:.3f}".format(Test_stat))
        print("p-value = {}".format(p_value))
        print("Hotelling's T^2 = {:.3f}".format(T_square))
        print("Do not reject the null hypothesis")

In [5]:
# Testing 결과

alpha = 0.05
u0 = [0,0]
hotelling_t_test(diff, u0, alpha)

Test Statistic = 1.270
p-value = 0.3135310532015403
Hotelling's T^2 = 2.735
Do not reject the null hypothesis


### b. Find the simultaneous confidence interval for 𝛿_1.

In [6]:
data = diff[['d1']]
n = len(diff)
p = len(diff.columns)
S = data.var()
alpha = 0.05
F = f.ppf(1-alpha, p, n-p, loc=0, scale=1)

lower = data.mean() - np.sqrt( (((n-1)*p) / (n-p)) * F * (S / n) )
upper = data.mean() + np.sqrt( (((n-1)*p) / (n-p)) * F * (S / n) )

print("Simultaneous confidence interval : lower = {}, upper = {}".format(lower[0], upper[0]))

Simultaneous confidence interval : lower = -546.0898558358173, upper = 159.2898558358173


### c. Perform a paired comparison using your python code in #4 of HW4. (I.e., do not use subtraction).

In [7]:
# Pyhton code in #4 of HW4

def profile_analysis_test(sample, C, alpha):
    
    # Calculate T square & Test statistic
    n = len(sample)
    S = sample.cov()
    sample_mean = sample.mean()
    df1 = C.shape[0] 
    df2 = n - C.shape[0] 
    t2 = n * (C @ sample.mean()).T @ np.linalg.inv(C @ S @ C.T) @ (C @ sample.mean()) # T square
    test_stat = t2 * ((n - df1) / ((n-1) * df1)) # Test statisitic
    
    # Calculate critical region
    c_region = f.ppf(1-alpha, df1, df2, loc=0, scale=1) * ( ((n-1) * df1) / (n - df1) )
    
    # p-value
    p_value = f.sf(test_stat, df1, df2)
    
    # Hypothesis Testing
    if t2 > c_region :
        print("Test statistic = {:.3f}".format(test_stat))
        print("t2 = {:.3f}".format(t2))
        print("p-value = {}".format(p_value))
        print("Reject the null hypothesis")
    else :
        print("Test statistic = {:.3f}".format(test_stat))
        print("t2 = {:.3f}".format(t2))
        print("p-value = {}".format(p_value))
        print("Do not reject the null hypothesis")

In [8]:
# Testing

a = np.identity(2)
b= -np.identity(2)
C = np.hstack((a,b))
alpha = 0.05
profile_analysis_test(lumber_2, C, alpha)

Test statistic = 1.270
t2 = 2.735
p-value = 0.31353105320153973
Do not reject the null hypothesis


### 2. Write a Python code to test the hypothesis

In [9]:
def two_sample_comparision(df1, df2, alpha) :
    n1 = len(df1)
    n2 = len(df2)
    df1_mean = df1.mean()
    df2_mean = df2.mean()
    S1 = df1.cov()
    S2 = df2.cov()
    S_p = ( (n1-1)*S1 + (n2-1)*S2 ) / ( n1 + n2 - 2 ) # Assume equal variance
    p = S_p.shape[0]
    alpha = alpha # parameter

    t2 = ( (n1*n2) / (n1+n2) ) * (df1_mean - df2_mean).T @ np.linalg.inv(S_p) @ (df1_mean - df2_mean)
    test_stat = ( (n1+n2-p-1) / (p*(n1+n2-2)) ) * t2

     # Calculate critical region
    c_region = f.ppf(1-alpha, p, n1+n2-p-1, loc=0, scale=1)

    # p-value
    p_value = f.sf(test_stat, p, n1+n2-p-1)

    # Hypothesis Testing
    if test_stat > c_region :
        print("Test statistic = {:.3f}".format(test_stat))
        print("t2 = {:.3f}".format(t2))
        print("p-value = {}".format(p_value))
        print("Reject the null hypothesis")
    else :
        print("Test statistic = {:.3f}".format(test_stat))
        print("t2 = {:.3f}".format(t2))
        print("p-value = {}".format(p_value))
        print("Do not reject the null hypothesis")

### 3. Table 6.9 contains measurements on the carapaces of 24 female and 24 male turtles. The data is in file ‘turtle.dat’. Assume equal covariances.

### a. Test the equality of the two population mean vectors at 𝛼 = 0.05. (Use your code in #2 above)

In [10]:
# 데이터 불러오기

turtle = pd.read_csv('turtle.dat', header=None, delim_whitespace=True)
turtle.columns = ['t1', 't2', 't3', 'sex']

# 남녀 데이터셋 나누기

turtle_female = turtle[turtle['sex'] == "female"][['t1', 't2', 't3']].reset_index(drop = True)
turtle_male = turtle[turtle['sex'] == "male"][['t1', 't2', 't3']].reset_index(drop = True)

In [11]:
# Testing

two_sample_comparision(turtle_male, turtle_female, 0.05)

Test statistic = 23.078
t2 = 72.382
p-value = 3.96672641348487e-09
Reject the null hypothesis


### b. Determine the lengths and directions for the axes of the 95% confidence ellipsoid for u1-u2.

In [12]:
S1 = turtle_female.cov()
S2 = turtle_male.cov()
n1 = len(turtle_female)
n2 = len(turtle_male)
p = len(turtle_female.columns)

S_p = ( (n1-1)*S1 + (n2-1)*S2 ) / ( n1 + n2 - 2 )
eigvals, eigvecs = np.linalg.eig(S_p)
F = f.ppf(1-alpha, p, n1+n2-p-1, loc=0, scale=1)
c_square = ( (p * (n1+n2-2)) / (n1+n2-p-1) ) * F

# Direction of the condfidence region : eigenvectors of pooled var-cov matrix
for i in range(len(turtle_male.columns)):
    print("Direction for Axes {} : {}".format(i+1, eigvecs[i]))
    

# Length of the confidence region
length_list = []
for i in range(len(turtle_male.columns)):
    length = np.sqrt(eigvals[i]) * np.sqrt( (1/n1 + 1/n2) * c_square ) * 2
    length_list.append(length)
print("Length of the confidence region : {}".format(length_list))

Direction for Axes 1 : [-0.82017422 -0.54070785 -0.18694725]
Direction for Axes 2 : [-0.49543231  0.83467139 -0.24056286]
Direction for Axes 3 : [-0.28611374  0.10468375  0.9524601 ]
Length of the confidence region : [35.844691855894354, 3.922499387311902, 2.684579712752479]


### c. Find the 95% simultaneous confidence intervals for the component mean differences.

In [13]:
lower_list = []
upper_list = []
for i in range(3):
    lower = (turtle_female.mean()[i] - turtle_male.mean()[i]) - np.sqrt(c_square) * np.sqrt((1/n1 + 1/n2) * S_p.iloc[i,i])
    upper = (turtle_female.mean()[i] - turtle_male.mean()[i]) + np.sqrt(c_square) * np.sqrt((1/n1 + 1/n2) * S_p.iloc[i,i])
    lower_list.append(lower)
    upper_list.append(upper)

    print("Simultaneous C.I. for mean difference of t{} : [{:.2f},{:.2f}]".format(i+1, lower_list[i], upper_list[i]))

Simultaneous C.I. for mean difference of t1 : [7.93,37.41]
Simultaneous C.I. for mean difference of t2 : [5.26,23.33]
Simultaneous C.I. for mean difference of t3 : [6.04,16.62]


### d. Find the 95% Bonferroni confidence intervals for the component mean differences.


In [14]:
from scipy.stats import t

t = t.ppf(1-(alpha/(2*p)), n1+n2-2)

lower_list = []
upper_list = []
for i in range(3):
    lower = (turtle_female.mean()[i] - turtle_male.mean()[i]) - t * np.sqrt((1/n1 + 1/n2) * S_p.iloc[i,i])
    upper = (turtle_female.mean()[i] - turtle_male.mean()[i]) + t * np.sqrt((1/n1 + 1/n2) * S_p.iloc[i,i])
    lower_list.append(lower)
    upper_list.append(upper)

    print("Bonferroni C.I. for mean difference of t{} : [{:.2f},{:.2f}]".format(i+1, lower_list[i], upper_list[i]))

Bonferroni C.I. for mean difference of t1 : [10.34,34.99]
Bonferroni C.I. for mean difference of t2 : [6.74,21.84]
Bonferroni C.I. for mean difference of t3 : [6.91,15.75]


### e. Test the equality of the two population covariance matrices at 𝛼 = 0.05 using your own code.

In [15]:
from scipy.stats import chi2
from numpy import log as ln

M = (n1+n2-2) * ln(np.linalg.det(S_p)) - (n1-1) * ln(np.linalg.det(S1)) - (n2-1) * ln(np.linalg.det(S2))
C_inverse = 1 - ( ( (2*(p**2) + 3*p -1) / (6*(p+1)) ) * ( (n1+n2-2) / ((n1-1)*(n2-1)) - (1 / (n1+n2-2)) ) )

df = (p*(p+1)) / 2

# Calculate critical region
c_region = chi2.ppf(1-alpha, df)

# p-value
p_value = chi2.sf(M*C_inverse, df)


# Hypothesis Testing
if M*C_inverse > c_region :
    print("p-value = {:.10f}".format(p_value))
    print("Reject the null hypothesis")
else :
    print("p-value = {}".format(p_value))
    print("Do not reject the null hypothesis")

p-value = 0.0006716087
Reject the null hypothesis


### f. Test the equality of the two population covariance matrices at 𝛼 = 0.05 using the Python package.

In [16]:
from statsmodels.stats import multivariate as mv
mv.test_cov_oneway([S1, S2], [24,24])

<class 'statsmodels.stats.base.HolderTuple'>
statistic = 3.8991762550674824
pvalue = 0.000678623715197216
statistic_base = 25.18423464462336
statistic_chi2 = 23.404913718644536
pvalue_chi2 = 0.0006716086888199255
df_chi2 = 6.0
distr_chi2 = 'chi2'
statistic_f = 3.8991762550674824
pvalue_f = 0.000678623715197216
df_f = (6.0, 15331.018867924493)
distr_f = 'F'
tuple = (3.8991762550674824, 0.000678623715197216)